In [1]:
#Candidate generation and pruning, python >3.7 req, 3.9.0 used
#Please have the 'playlists4000.txt' file in the same directory as this notebook.

import pandas as pd
import numpy as np

#Minimum support threshold
minsup = 50

#File Operations
fd = open("playlists4000.txt",mode='r',encoding='utf8',newline='\n')
raw = fd.read().splitlines()

#Create 1-itemset support count dictionary
counts = dict()
for line in raw:
    first, second = line.split(sep="::")
    x = counts.setdefault(second,0)
    counts[second] = x+1

#Create transaction dictionary
transactions = dict()

for line in raw:
    first, second = line.split(sep="::")
    transactions.setdefault(first, []).append(second)

In [3]:
#Generate frequent 1-itemsets
countscopy = counts.copy()

for x in countscopy.keys():
    if counts[x] < minsup:
        counts.pop(x)

In [10]:
import itertools

def candidate_gen_prune(items,k):
    #We are generating candidates based on F_k-1 X F_k-1 method.
    #Freq 1-itemsets are assumed to have been generated from filtering
    #initial support count dictionary based on minsup criterion.

    #Step 1: Candidate Generation
    gens = []
    for x in items:
        for y in items:
            gen = []
            #If generating 3-itemsets or more.
            if(k-2 > 0 and x[0:(k-2)] == y[0:(k-2)]):
                #List concatenation with iterable unpacking.
                gen = [*x[0:k],*y[(k-1):]]
            #If generating 2-itemsets.
            else:
                gen = [x,y]
            gens.append(gen)

    #Step 2: Candidate Pruning
    #Case if building 2-itemsets from 1-itemsets, every 1-itemset
    #is already frequent so we don't need to prune
    if(k < 3):
        return gens
    #Iterate over candidates.
    for s in gens:
        #Generate all subsets of length k-1 of each candidate.
        subsets = [list(l) for l in itertools.combinations(s,k)]
        for x in subsets:
            if(x not in items):
                gens.pop(s)
                break
    
    return gens

In [5]:
def support_counting(transactions: dict, counts: dict, k):
    for x in transactions.values():
        subsets = list(itertools.combinations(x,k))
        for y in counts:
            if( y in subsets ):
                counts[y] +=1
    
    return counts

In [ ]:
#Apriori Loop
k = 1
while(counts):
    k += 1
    cands = candidate_gen_prune(counts.keys(),k)
    counts = support_counting(transactions,counts,k)
    copy = counts.copy()
    keys = copy.keys()
    for x in keys:
        if counts[x] < minsup:
            counts.pop(x)